# NLP project 
## Dataset - Ag_news from HuggingFace


In [ ]:
# Instalation
!pip install nlp
!pip install datasets
!python -m spacy download
!pip install pystemmer

     |████████████████████████████████| 1.7MB 17.3MB/s 
     |████████████████████████████████| 245kB 43.7MB/s 
     |████████████████████████████████| 194kB 17.7MB/s 
     |████████████████████████████████| 112kB 14.4MB/s 
usage: spacy download [-h] [-d] model [pip_args [pip_args ...]]
spacy download: error: the following arguments are required: model
     |████████████████████████████████| 563kB 15.0MB/s 
  Created wheel for pystemmer: filename=PyStemmer-2.0.1-cp37-cp37m-linux_x86_64.whl size=425138 sha256=e9b6a40824710a013d5599c97f844f7cea90aa82c0c8e944bafffa9c36c1055d
  Stored in directory: /root/.cache/pip/wheels/f3/3c/11/ee323a09706e17a649c2730bd8819b06e887411ff7507acf7a
Successfully built pystemmer


In [ ]:
# import nlp
import datasets
from datasets import load_dataset
from spacy import load
# import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import tqdm
from typing import List, Tuple
from collections import Counter

# 1. Load and preprocess dataset

In [ ]:
dataset = load_dataset('ag_news')
dataset

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
dataset['train'][0]

{'label': 2,
 'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."}

In [ ]:
train_data = dataset["train"]
test_data = dataset["test"]
len(train_data), len(test_data)

(120000, 7600)

In [ ]:
train_data['text'][0]

"Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

# Preprocess data
## 1.1 Tokenize text using spacy library.

In [ ]:
spacy_nlp = load("en_core_web_sm")


def tokenize_spacy(text: str) -> List[str]:
  """Tokenize string with SpaCy. """

  disable = ["ner", "parser", "tagger", "lemmatizer", "tok2vec", "senter", "attribute_ruler"]
  tokens = spacy_nlp(text, disable=disable)
  return [str(token).lower() for token in tokens]


tokenize_spacy("I can't find my $25!")

['i', 'ca', "n't", 'find', 'my', '$', '25', '!']

## 1.2 Stemming

# Bag of embeddings

In [ ]:
import gensim.downloader
from gensim.models import KeyedVectors
import tqdm


glove = KeyedVectors.load("http://134.209.248.229:8081/glove-50k.bin")

In [ ]:
len(glove['cat']), glove['cat']

(200, array([ 0.14823  , -0.53152  , -0.25973  , -0.44095  ,  0.38555  ,
        -0.4114   , -0.56649  , -0.024739 , -0.2788   , -0.051034 ,
         0.3704   , -0.20881  , -0.30809  ,  0.3041   , -0.3933   ,
        -0.18335  , -0.59395  ,  0.57039  , -0.16796  ,  0.42368  ,
        -0.048687 ,  0.95404  ,  0.65675  , -0.61759  ,  0.41923  ,
        -0.42904  ,  0.2082   , -0.33186  , -0.0037637, -0.20592  ,
         0.18708  ,  0.06889  , -0.28935  , -0.7552   ,  0.29278  ,
        -0.43407  ,  0.034869 ,  0.081405 , -0.14095  , -0.34705  ,
         0.081686 ,  0.40679  , -0.27834  ,  0.93623  , -0.63422  ,
         0.77979  ,  0.29497  , -0.14873  ,  0.5111   ,  0.96405  ,
        -0.072559 , -0.064779 ,  0.20619  , -0.46259  ,  0.49824  ,
        -0.40328  , -0.92244  ,  0.4663   , -0.39433  ,  0.64215  ,
        -0.062508 , -0.71985  , -0.43478  ,  0.34995  , -0.6297   ,
        -0.21176  , -0.21179  , -0.57816  ,  0.84528  ,  0.097692 ,
         0.22564  , -0.2639   , -0.66842  ,

In [ ]:
import numpy as np
def bag_of_embeddings(dataset):
    X = []
    for doc in dataset:
        tokens = tokenize_spacy(doc["text"])
        token_vectors = []
        for token in tokens:
            if token in glove:
                token_vectors.append(glove[token])
        doc_vector = np.array(token_vectors).mean(axis=0)
        X.append(doc_vector)
    X = np.stack(X)
    y = np.array(dataset["label"])
    return (X, y)

In [ ]:
dataset = datasets.load_dataset("ag_news")

X_train_boe, y_tr = bag_of_embeddings(dataset["train"])
X_val_boe, y_v = bag_of_embeddings(dataset["test"])

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


# Linear regression using *embeddings*

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver="liblinear")
logreg.fit(X_train_boe, y_tr)
logreg_acc = logreg.score(X_val_boe, y_v)
logreg_acc

0.899

# SVM using *embeddings*

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=10000)
svm.fit(X_train_boe, y_tr)
svm_acc = svm.score(X_val_boe, y_v)
"SVM accuracy", svm_acc

('SVM accuracy', 0.8932894736842105)

In [ ]:
# Build vocab
STOP_WORDS = ["the", "and", "a", "of", "to", "is", "in", "that", "this", "was", "as", "with", "for", "you", "are", "it"]
bon_transform = CountVectorizer(
  max_features=5000,
  stop_words=STOP_WORDS,
  tokenizer=lambda doc: doc,
  ngram_range=[3,3],
  lowercase=False
)

X_tr_bon = bon_transform.fit_transform(train_data['text'])
X_te_bon = bon_transform.transform(test_data['text'])
y_tr = train_data['label']
y_te = test_data['label']

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'e', 'f', 'h', 'i', 'n', 'o', 'r', 's', 't', 'u', 'w', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
bow_transform = CountVectorizer(
  max_features=5000,
  stop_words=STOP_WORDS,
  tokenizer=lambda doc: doc,
  lowercase=False
)

X_tr_bow = bow_transform.fit_transform(train_data['text'])
X_te_bow = bow_transform.transform(test_data['text'])


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'e', 'f', 'h', 'i', 'n', 'o', 'r', 's', 't', 'u', 'w', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
from sklearn.feature_extraction import text

tfidf_transform = text.TfidfTransformer(norm=None)
X_tr_tfidf_bon = tfidf_transform.fit_transform(X_tr_bon)
X_te_tfidf_bon = tfidf_transform.transform(X_te_bon)

In [ ]:
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C, penalty="l2").fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model

In [ ]:
model_bow = simple_logistic_classify(X_tr_bow, y_tr, X_te_bow, y_te, 'Bag of words')
model_tfidf = simple_logistic_classify(X_tr_tfidf_bon, y_tr, X_te_tfidf_bon, y_te, 'tf-idf')
model_bon = simple_logistic_classify(X_tr_bon, y_tr, X_te_bon, y_te, 'Bag of ngrams')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Test Score with Bag of words features 0.5876315789473684


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Test Score with tf-idf features 0.8857894736842106
Test Score with Bag of ngrams features 0.8926315789473684


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Naive Bayes

In [ ]:
# Create model(naive bayes) and training. 
def naive_bayes(x_train, y_train, x_test, y_test, description):
  nb = MultinomialNB(alpha=0.9).fit(x_train, y_train)
  result = nb.score(x_test, y_test)
  print(f'Results for naive Bayes using {description} is: {result}')

In [ ]:
naive_bayes(X_tr_bon, y_tr, X_te_bon, y_te, 'Bag of ngrams')
naive_bayes(X_tr_bow, y_tr, X_te_bow, y_te, 'Bag of words')
naive_bayes(X_tr_tfidf_bon, y_tr, X_te_tfidf_bon, y_te, 'TF-IDF')

Results for naive Bayes using Bag of ngrams is: 0.8478947368421053
Results for naive Bayes using Bag of words is: 0.5296052631578947
Results for naive Bayes using TF-IDF is: 0.8553947368421052


# SVM

In [ ]:
from sklearn.svm import LinearSVC

def support_vector_m(x_train, y_train, x_test, y_test, description):
  svm = LinearSVC(max_iter=1000)
  svm.fit(x_train, y_train)
  svm_acc = svm.score(x_test, y_test)
  print("SVM accuracy", svm_acc)

In [ ]:
support_vector_m(X_tr_bon, y_tr, X_te_bon, y_te, 'Bag of ngrams')
support_vector_m(X_tr_bow, y_tr, X_te_bow, y_te, 'Bag of words')
support_vector_m(X_tr_tfidf_bon, y_tr, X_te_tfidf_bon, y_te, 'TF-IDF')

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


SVM accuracy 0.8951315789473684
SVM accuracy 0.5473684210526316
SVM accuracy 0.878421052631579
